### Install required libraries to collect the data. 
We use praw to collect data from subreddit.

In [ ]:
import pymongo
from pymongo import MongoClient
import praw
from praw.models import MoreComments
import pandas as pd
from bs4 import BeautifulSoup
import re

Connect to mongo client

In [11]:
client = MongoClient('mongodb://localhost:27017/')
db = client["reddit"]

Connect to reddit api using secret keys from your account.

In [12]:
### Reddit API Credentials
c_id = ""               # Personal Use Script (14 char)
c_secret = ""           # Secret key (27 char)
u_a = ""               # App name
usrnm = ""      # Reddit username
passwd = ""  

In [13]:
reddit = praw.Reddit(client_id = c_id,
                     client_secret = c_secret,
                     user_agent = u_a,
                     username = usrnm,
                     password = passwd)


## Data Collection
search for 200 posts through the subreddit for each flair you want. Use the paramters you find useful such as title, body, upvotes, comments etc. We are collecting the top 10 comments from each post. A balanced dataset of 200 posts for each flair will be created.
The following data is collected from subreddit [r/india](https://www.reddit.com/r/india/) on 15 April, 2020.

In [ ]:
#List of all the flairs. These will be the keys in classification.
flairs = [ "AskIndia", "Food","Non-Political",
          "Photography", "Science/Technology","Policy/Economy",
          "Politics", "Business/Finance","Sports"]

subreddit = reddit.subreddit('india')

for flair in flairs:
    print (flair)
    sub_posts = subreddit.search(f"flair_name:{flair}",limit=200)
    for post in sub_posts:
        posts = {
        "title":str(post.title),
        "score":str(post.score),
        "id":str(post.id),
        "body":str(post.selftext),
        "author":str(post.author),
        "flair":str(flair),	
        "url":str(post.url),
        "comms_num":str(post.num_comments),
        "created":str(post.created),
        }

#  Only top ten comments and their authors are considered. 
        post.comments.replace_more(limit=None)
        comment = str()
        authors = str()
        count = 0
        for top_comment in post.comments:
            comment = comment + ' ' + top_comment.body
            authors = authors + ' ' + str(top_comment.author)
            count+=1     
            if(count > 10):
                break

        posts["comment"] = str(comment)
        posts["authors"] = str(authors)
        result = db.posts.insert_one(posts)

## Save data in a csv file

In [54]:
collection = db.posts
posts = pd.DataFrame(list(collection.find()))

del posts['_id']

posts['title'] = posts['title'].apply(lambda x:str(x))
posts['body'] = posts['body'].apply(lambda x:str(x))
posts['comment'] = posts['comment'].apply(lambda x:str(x))

combined_features = posts["title"] + posts["comment"] + posts["url"] + posts["body"]
posts = posts.assign(combined_features = combined_features)

#saving the csv file
posts.to_csv('./data.csv',index=False)

,title,score,id,body,author,flair,url,comms_num,created,comment,authors,combined_features
0,"Lost my Job, Sick Mother and Paralysed Dad, In...",1042,g014wc,Hi....It's really tough time for everyone. I r...,sanand_satwik,AskIndia,https://www.reddit.com/r/india/comments/g014wc...,132,1586742148.0,I'm a freelancer. Don't listen to the idiots ...,hashedram diabapp xataari Aashayrao sarcrasti...,"Lost my Job, Sick Mother and Paralysed Dad, In..."
1,Why does the government come with a begging bo...,647,fxofyu,"We have floods, terrorist attacks, famines due...",TWO-WHEELER-MAFIA,AskIndia,https://www.reddit.com/r/india/comments/fxofyu...,205,1586448261.0,I don't understand why they don't use money f...,Kinky-Monk ak32009 fools_eye None DwncstSheep...,Why does the government come with a begging bo...
2,Mother's condition is going worse due to hepat...,756,g0zlly,"Hi folks, I really appreciate the warm respons...",sanand_satwik,AskIndia,https://www.reddit.com/r/india/comments/g0zlly...,92,1586870771.0,If anyone knows who is influential on Twitter...,AlternativeDrop6 TheRobotsHaveCome lanky32 pl...,Mother's condition is going worse due to hepat...
3,Men who are 30+ and have decided not to get ma...,264,fvy95j,The corona virus has given me some time to thi...,indianoogler,AskIndia,https://www.reddit.com/r/india/comments/fvy95j...,211,1586206585.0,Plan your finances. Work and enjoy in your ow...,RedDevil-84 congratsindia khushraho kingof-po...,Men who are 30+ and have decided not to get ma...
4,"r/India: If money is no bar, would you prefer ...",133,g0igt7,Seems like everybody here is very critical of ...,cataglottis,AskIndia,https://www.reddit.com/r/india/comments/g0igt7...,156,1586811549.0,"if money is not a thing, i would choose india...",_azadak 1209naveen reti_opening kokishinkiba ...,"r/India: If money is no bar, would you prefer ..."
